# AWS IoT Authentication and Authorization


In [2]:
store -r home
home

SyntaxError: invalid syntax (<ipython-input-2-4c432f9e50ad>, line 1)

Create a Thing, a unique name helps to make references more clear.
Any computer can be a thing, lets use this notebook instance as our thing to begin.

In [ ]:
from datetime import datetime
HM = datetime.now().strftime('%H%M')
%store HM

In [ ]:
thing_name = f"NotebookThing{HM}"
thing_name

In [ ]:
! aws iot create-thing --thing-name {thing_name}

Create a certificate and keys to authenticate your thing. 

In [ ]:

! mkdir "{home}/aws_iot"

certificate_file = f"{home}/aws_iot/certificate{HM}.pem"
public_key = f"{home}/aws_iot/public-key{HM}.pem"
private_key = f"{home}/aws_iot/private-key{HM}.pem"


%store certificate_file
%store public_key
%store private_key

In [ ]:
certificateArn = ! aws iot create-keys-and-certificate \
  --set-as-active \
  --certificate-pem-outfile {certificate_file} \
  --public-key-outfile {public_key} \
  --private-key-outfile {private_key} \
  --query certificateArn \
  --output text
certificate_arn = certificateArn.s
certificate_arn

Download the root Certification Authority used by AWS IoT


In [ ]:
ca_file = f"{home}/aws_iot/ca{HM}.pem"
%store ca_file
! wget https://www.symantec.com/content/en/us/enterprise/verisign/roots/VeriSign-Class%203-Public-Primary-Certification-Authority-G5.pem -O {ca_file}

Double check that you have the 4 files required for TLS mutual authentication used by AWS IoT
1. Private Key
1. Public Key
1. Certificate
1. Root CA

In [ ]:
! printf "Private Key {private_key} " && [ -f {private_key} ] && printf "found" || printf "not found"
! printf "\nPublic Key {public_key} " && [ -f {public_key} ] && printf "found" || printf "not found"
! printf "\nCertificate {certificate_file} " && [ -f {certificate_file} ] && printf "found" || printf "not found"
! printf "\nCertification Authority {ca_file} " && [ -f {ca_file} ] && printf "found" || printf "not found"

Create an IAM authorization policy stating what is going to be allowed.

In [ ]:
#TODO: Use a more restrictive policy
policy_document = '''{
  "Version": "2012-10-17",
  "Statement": [{
    "Effect": "Allow",
    "Action": "iot:*",
    "Resource": "*"}]
}'''

In [ ]:
policy_name = f"NotebooksPolicy{HM}"
policyArn = ! aws iot create-policy \
    --policy-name {policy_name} \
    --policy-document '{policy_document}' \
    --query policyArn \
    --output text
policyArn.s

Attach certificate to thing and policy

In [ ]:
! aws iot attach-policy \
    --policy-name {policy_name} \
    --target {certificate_arn}

In [ ]:
! aws iot attach-thing-principal \
  --thing-name {thing_name} \
  --principal {certificate_arn}

# Connecting to AWS IoT

AWS IoT provides account-specific endpoints for your things to connect:

In [ ]:
endpoint_address = !aws iot describe-endpoint --query endpointAddress --output=text
endpoint_address = endpoint_address.s
%store endpoint_address
endpoint_address

Ready to go! Create your MQTT client instance:

In [ ]:
from AWSIoTPythonSDK.MQTTLib import AWSIoTMQTTClient

client_id=f"NotebookCient{HM}"
mqtt = AWSIoTMQTTClient(client_id)
mqtt.configureEndpoint(endpoint_address, 8883)
mqtt.configureCredentials(ca_file, private_key, certificate_file)
mqtt.configureConnectDisconnectTimeout(10)
mqtt

# Receiving and Sending Messages

Connect to the service to send and receive messages:

In [ ]:
mqtt.connect()

Subscribe to a topic:

In [ ]:
topic_name ="notebook-topic"
%store topic_name
subscribe_qos = 1
def custom_callback(client, userdata, message):
    print("\nReceived a new message: \n")
    print(message.payload.decode("utf-8") )
    print("\n--------------\n")
    
mqtt.subscribe(topic_name, subscribe_qos, custom_callback)

Publish to a topic:

In [ ]:
import uuid
import json
import datetime 
import random
import time
def genPayload():
    names = ["Sarah Jane", "Rory Williams", "Amy Pond", "Jack Harkness", "Clara Oswald", "Martha Jones", "Rose Tyler"] 
    name = random.choice(names)
    id = str(uuid.uuid4())
    now = int(time.time()*1000)
    noise = round(40 * random.random());
    data = {
        "id" : id ,
        "name" : name,
        "sistolic" : 110 + noise,
        "diastolic" : 70 + noise,
        "time_stamp" : now
    }
    payload = json.dumps(data)
    return payload
payloads = []
for x in range(100):
    payloads.append(genPayload())
%store payloads
payload = random.choice(payloads)
%store payload
payload

In [ ]:
publish_qos = 0
mqtt.publish(topic_name, payload , publish_qos)

The subscribe_qos and publish_qos arguments refer to the quality of service parameter of the MQTT protocol. Selecting the QoS level is a tradeoff between delivery guarantee and performance, see the following resources for more infomation:

https://www.hivemq.com/blog/mqtt-essentials-part-6-mqtt-quality-of-service-levels

https://docs.aws.amazon.com//iot/latest/developerguide/protocols.html

All working fine, done for now, disconnect.

In [ ]:
mqtt.disconnect()